In [1]:
import alpaca_trade_api as tradeapi
import threading
import time
import datetime
# config.py contains my keys
from config import *
import numpy as np
import pandas as pd
import edhec_risk_kit as erk
import bs4 as bs
import pickle
import requests
import yfinance as yf
APCA_API_BASE_URL = "https://paper-api.alpaca.markets"
%load_ext autoreload
%autoreload 2

In [2]:
# function from https://pythonprogramming.net/sp500-company-list-python-programming-for-finance/
# slightly altered
def get_sp500_stocks():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
    
    tickers = [x.replace('\n', '') for x in tickers]
    
    # drop BKR, BRK.B and BF.B
    # Alpaca doesn't like BKR
    # Yahoo finance doesn't like BRK.B and BF.B
    tickers = [x for x in tickers if x != 'BKR']
    tickers = [x for x in tickers if x != 'BRK.B']
    tickers = [x for x in tickers if x != 'BF.B']
    tickers = ' '.join(tickers)
    
    return tickers

tickers = get_sp500_stocks()

### Download stock close price data for 2019

In [3]:
data = yf.download(tickers, start='2019-01-01', end='2019-12-31')

[*********************100%***********************]  502 of 502 completed


### Calculate daily returns

In [4]:
rets = data['Adj Close'].pct_change()
# first drop columns where all of the values are NA
rets.dropna(axis=1, inplace=True, how='all')
# then drop the first row (where all the values are NA)
rets.dropna(axis=0, inplace=True, how='all')

### Calculate weights of the global minimum variance portfolio

In [5]:
weights = np.round(erk.gmv(rets.cov()), 5)

### Connect to the API & retrieve the amount of cash I have in my paper account

In [6]:
alpaca = tradeapi.REST(API_KEY, API_SECRET, APCA_API_BASE_URL, 'v2')
cash = int(float(alpaca.get_account().cash))

### Cancel any existing open orders

In [7]:
orders = alpaca.list_orders(status='open')
for order in orders:
    alpaca.cancel_order(order.id)

### Calculate the dollar amount I want to buy of each stock

In [8]:
dollar_amounts = weights*cash

In [9]:
# function from https://github.com/alpacahq/alpaca-trade-api-python/blob/master/examples/long-short.py
# slightly altered
def submitOrder(qty, stock, side):
    if qty > 0:
        try: 
            alpaca.submit_order(stock, qty, side, 'market', 'day')
            print('Submitted order to ' + side + ' ' + str(qty) + ' shares(s) of ' + stock)
        except:
            print('Order failed to submit: ' + side + ' of ' + str(qty) + ' share(s) of ' + stock)
    else:
        print('Quantity for ' + stock + ' is zero b/c dollar_amount < share_price')

### Calculate quantity of shares of each stock that I'm buying & submit orders

In [10]:
stocks = list(data['Adj Close'].columns)
for stock, amount in zip(stocks, dollar_amounts):
    try:
        # If the dollar amount that I want to buy of a stock is greater than $1,
        # attempt to retrieve the share price & submit an order
        if amount > 1:
            qty = int(amount//alpaca.get_barset(stock, 'minute', 1)[stock][0].c)
            submitOrder(qty, stock, 'buy')
    except:
        print("Couldn't pull stock data for " + stock + " from Alpaca")

Submitted order to buy 4 shares(s) of ABMD
Submitted order to buy 4 shares(s) of AGN
Submitted order to buy 492 shares(s) of AMCR
Quantity for AZO is zero b/c dollar_amount < share_price
Submitted order to buy 2 shares(s) of BIIB
Submitted order to buy 52 shares(s) of CBOE
Submitted order to buy 18 shares(s) of CHD
Submitted order to buy 1 shares(s) of CHTR
Submitted order to buy 2 shares(s) of CMA
Submitted order to buy 11 shares(s) of CTSH
Submitted order to buy 40 shares(s) of CTXS
Submitted order to buy 1 shares(s) of DG
Submitted order to buy 9 shares(s) of DISCK
Submitted order to buy 31 shares(s) of DLTR
Submitted order to buy 6 shares(s) of ES
Submitted order to buy 115 shares(s) of EVRG
Submitted order to buy 44 shares(s) of EXR
Submitted order to buy 6 shares(s) of FITB
Submitted order to buy 84 shares(s) of FOXA
Submitted order to buy 28 shares(s) of HP
Submitted order to buy 108 shares(s) of HRB
Submitted order to buy 26 shares(s) of HSY
Submitted order to buy 2 shares(s) o